Retrieve time series of market cap to calculate number of stocks, exchange market cap and primary stocks

In [5]:
# Libraries
import eikon as ek
import csv
import pandas as pd
from tqdm import tqdm

In [6]:
# Credentials
ek.set_app_key("YOUR_KEY")

# Circumvence the problem with 404 Bad request
# https://community.developers.refinitiv.com/questions/45723/why-eikonerror-error-code-400-backend-error-400-ba.html
ek.set_timeout(10000000)

In [7]:
# Screener: exclude BETX [One exchange has to be excluded otherwise eikon does not fetch the data] from SCREENER
# The rics we want to have information for
ric = "SCREEN(U(IN(Equity(active or inactive,public,primary))/*UNV:Public*/), NOT_IN(TR.ExchangeMarketIdCode,""BETX""), CURN=USD)"

In [8]:
field = ["TR.CommonName"]

In [10]:
df, err = ek.get_data(ric, field)

In [11]:
# Slice the data for rics and create a list for ek.get_data()
rics = list(df["Instrument"])

In [19]:
df["Instrument"].count()

73353

In [11]:
# From Data Item
field = ["TR.CompanyMarketCap.Date; TR.CompanyMarketCap"]
start = "20000131"
end   = "20200131"

In [12]:
ls_RIC = {}

In [13]:
# Eikon has a limit of 5000 rics per retrieval. Choose 3000 to see a progress in the for loop for ek.get_data().
interval = 3000
i = 3000
ric_set = range(round(df3["Instrument"].count()/ interval))

for rics in tqdm(ric_set):
    ls_RIC["RIC_SET_{0}".format(rics)] = list(df3['Instrument'][i - interval:i].astype('str'))
    i += interval

100%|██████████| 24/24 [00:00<00:00, 459.58it/s]


In [14]:
ls_RIC_keys = list(ls_RIC.keys())

In [15]:
data_dict = {}

In [ ]:
# The for-loop is splitted in three chunks because it crashes with 400 error bad request (An internal Eikon error)
# https://community.developers.refinitiv.com/questions/45723/why-eikonerror-error-code-400-backend-error-400-ba.html

for keys in tqdm(ls_RIC_keys[:8]):
    data_dict["data_{0}".format(keys)], err = ek.get_data(ls_RIC.get(keys), field, 
                                                          {"SDate":start, 
                                                           "EDate":end, 
                                                           "Frq":"Y", 
                                                           "curn":"USD", 
                                                           # in Millions
                                                           "scale":"6"
                                                          })

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
for keys in tqdm(ls_RIC_keys[8:16]):
    data_dict["data_{0}".format(keys)], err = ek.get_data(ls_RIC.get(keys), field, 
                                                          {"SDate":start, 
                                                           "EDate":end, 
                                                           "Frq":"Y", 
                                                           "curn":"USD", 
                                                           # in Millions
                                                           "scale":"6"
                                                          })

In [ ]:
for keys in tqdm(ls_RIC_keys[16:]):
    data_dict["data_{0}".format(keys)], err = ek.get_data(ls_RIC.get(keys), field, 
                                                          {"SDate":start, 
                                                           "EDate":end, 
                                                           "Frq":"Y", 
                                                           "curn":"USD", 
                                                           # in Millions
                                                           "scale":"6"
                                                          })

In [ ]:
# Merge the dictionary to get one data frame to write as csv later on.
df = pd.concat(data_dict, axis = 0)

In [ ]:
df = df.reset_index()

In [ ]:
df = df.dropna()

In [ ]:
df = df.drop(columns = ["level_0", "level_1"])

In [ ]:
df.to_csv(r'market_cap_over_time.csv')